<a href="https://colab.research.google.com/github/sudarshan-koirala/youtube-stuffs/blob/main/langchain/Quickstart_Guide(Part_2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Quickstart Guide (part 2)
### [Youtube Video Covering this notebook](https://youtu.be/-nCW1b1BTxU)
# Building a Language Model Application: Chat Models
- https://python.langchain.com/en/latest/getting_started/getting_started.html
- This 2nd part of quickstart guide provides you a quick walkthrough about using chat models instead of LLMs.
- Chat models are a variation on language models. While chat models use language models under the hood, the interface they expose is a bit different: rather than expose a “text in, text out” API, they expose an interface where “chat messages” are the inputs and outputs.

## Installation

In [ ]:
!pip install langchain openai huggingface_hub watermark --quiet

In [ ]:
%load_ext watermark
%watermark -a "Sudarshan Koirala"

## Environment Setup
- Using LangChain will usually require integrations with one or more model providers, data stores, apis, etc.

- Good to know: <font color="red">OpenAI's API (its not free)</font>

**Get api keys (you need to create account for both to access the api keys)**

- Get OpenAI api key: https://platform.openai.com/account/api-keys

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "OPENAI_API_KEY"

📝NOTE: LangChain provides many modules that can be used to build language model applications. Modules can be combined to create more complex applications, or be used individually for simple applications.

## Get Message Completions from a Chat Model
- We can get chat completions by passing one or more messages to chat model and response will be message.
- <font color="orange"> The types of messages currently supported in LangChain are `AIMessage`, `HumanMessage`, `SystemMessage` and `ChatMessage` – `ChatMessage` takes in an arbitrary role parameter. Most of the time, you’ll just be dealing with `HumanMessage`, `AIMessage`, and `SystemMessage`. </font>


In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)

- The temperature argument in the OpenAI LLM wrapper is used to <font color="red">control the level of randomness in the generated text.</font>
- <font color="cyan">A higher temperature value will result in more diverse and unpredictable text, while a lower temperature value will result in more conservative and predictable text. </font> 
- The default value for temperature is 1.0, and valid values range from 0.0 to 1.0.

In [ ]:
# we can now get completions by passing in a single message
chat([HumanMessage(content="Translate this sentence from English to Nepali. I love programming.")])

AIMessage(content='मलाई प्रोग्रामिङ्ग मन पर्छ।', additional_kwargs={})

In [ ]:
# we can pass multiple messages for OpenAI’s gpt-3.5-turbo and gpt-4 models
messages = [
    SystemMessage(content="You are a helpful assistant that translates English to Nepali."),
    HumanMessage(content="Translate this sentence from English to Nepali. I love programming.")
]
chat(messages)

AIMessage(content='मलाई प्रोग्रामिङ मन पर्छ।', additional_kwargs={})

You can go one step further and generate completions for multiple sets of messages using `generate`. This returns an `LLMResult` with an additional `message` parameter

In [ ]:
batch_messages = [
    [
        SystemMessage(content="You are a helpful assistant that translates English to Nepali."),
        HumanMessage(content="Translate this sentence from English to Nepali. I love programming.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to Nepali."),
        HumanMessage(content="Translate this sentence from English to Nepali. I love artificial intelligence.")
    ],
]
result = chat.generate(batch_messages)
result

LLMResult(generations=[[ChatGeneration(text='मलाई प्रोग्रामिङ मन पर्छ।', generation_info=None, message=AIMessage(content='मलाई प्रोग्रामिङ मन पर्छ।', additional_kwargs={}))], [ChatGeneration(text='मलाई कृत्रिम बुद्धिमत्ता मन पर्छ।', generation_info=None, message=AIMessage(content='मलाई कृत्रिम बुद्धिमत्ता मन पर्छ।', additional_kwargs={}))]], llm_output={'token_usage': {'prompt_tokens': 77, 'completion_tokens': 54, 'total_tokens': 131}, 'model_name': 'gpt-3.5-turbo'})

In [ ]:
# from the output we get we can even extract specific things like token usage
result.llm_output['token_usage']

{'prompt_tokens': 77, 'completion_tokens': 54, 'total_tokens': 131}

## Chat Prompt Templates
- <font color="orange">Instead of hard-coding thet text we want to ask, similar to LLMs, we can use make use of templating by using a `MessagePromptTemplate`.</font> 
- We can build a `ChatPromptTemplate` from one or more `MessagePromptTemplate`s.
- We can use `ChatPromptTemplate`’s `format_prompt` – this returns a `PromptValue`, which we can convert to a string or `Message` object, depending on whether you want to use the formatted value as input to an llm or chat model.
- For simplicity, there is a `from_template` method exposed on the template which makes our task lot easier.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

# get a chat completion from the formatted messages
chat(chat_prompt.format_prompt(input_language="English", output_language="Nepali", text="I love programming.").to_messages())

AIMessage(content='मलाई प्रोग्रामिङ्ग मन पर्छ।', additional_kwargs={})

## Chains and Chat Models
- In real application, Using LLM in isolation is OK for some applications but in most of the cases it requires chaining. And chaining with PromptTemplate might be a neccessity. 
- A chain in LangChain is made up of links, which can be either primitives like LLMs or other chains
- <font color="red">Similar to what we did for LLMs in the previous video, extending the previous example, we can construct an LLMChain which takes user input, formats it with a PromptTemplate, and then passes the formatted response to an LLM.</font>

In [ ]:
#ChatPromptTemplate??

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain import LLMChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatOpenAI(temperature=0)

template="You are a helpful assistant that translates {input_language} to {output_language}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="{text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

chain = LLMChain(llm=chat, prompt=chat_prompt)
chain.run(input_language="English", output_language="Nepali", text="I love programming.")

'मलाई प्रोग्रामिङ्ग मन पर्छ।'

## Agents and Chat Models
- So, far what we did was to run the chains in a predetermined order.
- <font color ="orange"> Agents can also be used with chat models, you can initialize one using `AgentType`. `CHAT_ZERO_SHOT_REACT_DESCRIPTION` as the agent type.</font>

In order to load agents, understanding the following concepts is crucial.

- Tool: A function that performs a specific duty. This can be things like: Google Search, Database lookup, Python REPL, other chains.

- Chat Models: The chat models powering the agent.

- Agents: Agents involve an LLM making decisions about which actions to take, taking that Action, seeing an observation, and repeating that until its done.

Agents: For a list of supported agents and their specifications, see [here](https://python.langchain.com/en/latest/modules/agents/agents.html).

Tools: For a list of predefined tools and their specifications, see [here](https://python.langchain.com/en/latest/modules/agents/tools.html).

<font color="red">For this example, you will also need to install the [SerpAPI Python package](https://pypi.org/project/google-search-results/).</font>



In [ ]:
!pip install google-search-results --quiet

In [ ]:
# https://serpapi.com/
import os
os.environ['SERPAPI_API_KEY'] = "SERPAPI_API_KEY"

<font color="yellow">LLM Math is a Python package that showcases using LLMs and Python REPLs to do complex word math problems. It allows you to input a math problem in natural language and get the answer in return.</>

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

# First, let's load the language model we're going to use to control the agent.
chat = ChatOpenAI(temperature=0)

# Next, let's load some tools to use. Note that the `llm-math` tool uses an LLM, so we need to pass that in.
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi", "llm-math"], llm=llm)


# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, chat, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# Now let's test it out!
agent.run("Who is the president of Finland ? What is 2+2 ?")



> Entering new AgentExecutor chain...
Question: Who is the president of Finland? What is 2+2?
Thought: The first question requires a search, while the second question requires a calculator.
Action:
```
{
  "action": "Search",
  "action_input": "President of Finland"
}
```

Observation: Sauli Niinistö
Thought:For the second question, I need to use the calculator tool.
Action:
```
{
  "action": "Calculator",
  "action_input": "2+2"
}
```


Observation: Answer: 4
Thought:I now know the final answer to the original input question.
Final Answer: Sauli Niinistö, 4.

> Finished chain.


'Sauli Niinistö, 4.'

## Memory: Add State to Chains and Agents
- So far, all the chains and agents we’ve gone through have been stateless. But often, you may want a chain or agent to have some concept of “memory” so that it may remember information about its previous interactions.
- For example, while designing a chatbot you want it to remember previous message or previous several messages.
- Short-term memory
- Long-term-memory (remembering key pieces of information over time)
- <font color="red"> You can use Memory with chains and agents initialized with chat models. The main difference between this and Memory for LLMs is that rather than trying to condense all previous messages into a string, we can keep them as their own unique memory object.</font>


In [ ]:
from langchain.prompts import (
    ChatPromptTemplate, 
    MessagesPlaceholder, 
    SystemMessagePromptTemplate, 
    HumanMessagePromptTemplate
)
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI
from langchain.memory import ConversationBufferMemory

prompt = ChatPromptTemplate.from_messages([
    SystemMessagePromptTemplate.from_template("The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know."),
    MessagesPlaceholder(variable_name="history"),
    HumanMessagePromptTemplate.from_template("{input}")
])

llm = ChatOpenAI(temperature=0)
memory = ConversationBufferMemory(return_messages=True)
conversation = ConversationChain(memory=memory, prompt=prompt, llm=llm)

conversation.predict(input="Hi there!")

'Hello! How can I assist you today?'

In [ ]:
conversation.predict(input="I'm doing well! Just having a conversation with an AI.")

"That sounds like fun! I'm happy to chat with you. Is there anything specific you'd like to talk about?"

In [ ]:
conversation.predict(input="Tell me about yourself.")

"Sure! I am an AI language model created by OpenAI. I was trained on a large dataset of text from the internet, which allows me to understand and generate human-like language. I can answer questions, provide information, and even have conversations like this one. Is there anything else you'd like to know about me?"

## Conclusion
- <font color="red">The decision to use a chat model or an LLM would depend on the specific task you are trying to accomplish. Chat models are designed to have structured conversations with humans, so they would be useful for tasks like creating chatbots or customer service agents. On the other hand, LLMs are more general language models that can be used for a wide range of tasks, such as language translation, text generation, or summarization.</font>

- <font color="orange">In general, if the task involves interacting with humans in a conversational way, a chat model would be the better choice. But if the task involves generating or processing large amounts of text, an LLM would be more appropriate. It's also worth noting that LLMs can be used in conjunction with other models, such as text embedding models, to improve their performance on specific tasks.</font>